### AI agent Tool Basic

In [8]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import SQLDatabase
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

load_dotenv()


# 1. llm 설정

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#2. postgresql 연결 설정

db = SQLDatabase.from_uri(f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}")
## 3. SQL 실행 도구 정의

@tool
def run_sql_query(query: str) -> str:
    """SQL 쿼리를 실행하고 결과를 반환합니다."""
    db_tool = QuerySQLDataBaseTool(db=db)
    return db_tool.invoke({"query": query})
  
tools = [run_sql_query]

# 4. 에이전트 생성

prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 데이터베이스 전문가입니다. 사용자의 질문을 SQL 쿼리로 변환해 실행하세요."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}") # 이전도구 호출기록
])

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True)
# 5. 실행 예시

if __name__ == "__main__":
    agent_executor.invoke({"input": "langchain_pg_embedding 테이블의 모든 이름을 보여줘."})



/Users/hyunjunson/Project/green-hat/backend/ml-pipline/yes/envs/oracle-langchain/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:159: SAWarning: Did not recognize type 'vector' of column 'vector_title'
  self._metadata.reflect(
/Users/hyunjunson/Project/green-hat/backend/ml-pipline/yes/envs/oracle-langchain/lib/python3.11/site-packages/langchain_community/utilities/sql_database.py:159: SAWarning: Did not recognize type 'vector' of column 'embedding'
  self._metadata.reflect(




> Entering new AgentExecutor chain...

Invoking: `run_sql_query` with `{'query': 'SELECT name FROM langchain_pg_embedding;'}`


Error: (psycopg2.errors.UndefinedColumn) column "name" does not exist
LINE 1: SELECT name FROM langchain_pg_embedding;
               ^

[SQL: SELECT name FROM langchain_pg_embedding;]
(Background on this error at: https://sqlalche.me/e/20/f405)

/var/folders/qb/k2smljtx419g49xbkmv9zs700000gn/T/ipykernel_37121/3588796086.py:25: LangChainDeprecationWarning: The class `QuerySQLDataBaseTool` was deprecated in LangChain 0.3.12 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-community package and should be used instead. To use it run `pip install -U :class:`~langchain-community` and import as `from :class:`~langchain_community.tools import QuerySQLDatabaseTool``.
  db_tool = QuerySQLDataBaseTool(db=db)



Invoking: `run_sql_query` with `{'query': 'SELECT * FROM langchain_pg_embedding LIMIT 1;'}`


[('abe4cf46-9de3-41d2-8d1e-0c50f65ef7a2', UUID('e1d674a4-221b-491b-bf2d-5cee452c5255'), '[0.013119346,0.0360535,-0.0069658794,0.033418655,0.03517522,-0.006740819,-0.03622916,0.081812024,0.0027542394,-0.03862248,0.015622453,-0.057527512,-0.03642677,-0.0580984,-0.0043447544,0.015973765,-0.052345645,-0.030059224,-0.006762776,-0.049096,-0.027731774,0.04084014,-0.00053588965,-0.0028653971...', '2025년10월호인공지능 산업의 최신 동향', {'page': 0, 'author': 'dj', 'source': 'data/SPRi AI Brief_10월호_산업동향_1002_F.pdf', 'creator': 'Hancom PDF 1.3.0.505', 'moddate': '2025-10-02T12:57:42+09:00', 'producer': 'Hancom PDF 1.3.0.505', 'page_label': '1', 'pdfversion': '1.4', 'total_pages': 29, 'creationdate': '2025-10-02T12:57:42+09:00'})]
Invoking: `run_sql_query` with `{'query': 'SELECT * FROM langchain_pg_embedding;'}`


[('abe4cf46-9de3-41d2-8d1e-0c50f65ef7a2', UUID('e1d674a4-221b-491b-bf2d-5cee452c5255'), '[0.013119346,

In [9]:
from dotenv import load_dotenv
import os
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain.agents import tool
from langchain_core.prompts import ChatPromptTemplate
from langchain_community.utilities import GoogleSerperAPIWrapper

# SERPER_API 설치 필요 -> https://serper.dev

load_dotenv()
@tool
def google_search(query: str) -> str:
    """Google 검색을 수행하고 결과를 반환합니다."""
    search = GoogleSerperAPIWrapper()
    return search.run(query)

@tool
def calculator(expression: str) -> str: 
    """간단한 수학 계산을 수행합니다."""
    try:
        result = eval(expression)
        return f"계산결과 : {result}"
    except Exception as e:
        return f"계산 오류: {str(e)}"

# llm 설정
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

#도구 리스트
tools = [google_search, calculator]

#프롬프트 템플릿
prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 유능한 AI AGENT입니다. 사용자의 질문에 최선을 다해 답변하세요. 필요시 도구를 사용하세요."),
    ("human", "{input}"),
    ("placeholder", "{agent_scratchpad}") # 이전도구 호출기록
])

# 에이전트 생성

agent = create_tool_calling_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors="Check your output format and try again" #출력형식 오류시 재시도 
    )

# 실행 예시
if __name__ == "__main__":
    # 1. 구글 검색
    print("== 구글 검색 예시 ==")
    print("질문 1: 2024년 노벨 물리학상 수상자는?")
    print("=" * 50)
    response1 = agent_executor.invoke({"input": "2024년 노벨 물리학상 수상자는?"})
    print(f"\n답변: {response1['output']}\n")
    
    #2. 계산기
    print("== 계산기 예시 ==")
    print("질문 2: 12345 곱하기 67890은?")
    print("=" * 50)
    print("=" * 50)
    response2 = agent_executor.invoke({"input": "12345 곱하기 67890은?"})
    print(f"\n답변: {response2['output']}\n")
    
    #3. 검색과 계산의 조합
    print("=" * 50)
    print("질문 3: 현재 비트코인 가격을 알려주고 10개 구매시 비용은?")
    print("=" * 50)
    response3 = agent_executor.invoke({"input": "현재 비트코인 가격을 검색하고, 10개를 구매하면 얼마인지 계산해 주세요."})
    print(f"\n답변: {response3['output']}\n")
    

== 구글 검색 예시 ==
질문 1: 2024년 노벨 물리학상 수상자는?


> Entering new AgentExecutor chain...

Invoking: `google_search` with `{'query': '2024 Nobel Prize in Physics winner'}`


The Nobel Prize in Physics 2024 was awarded jointly to John J. Hopfield and Geoffrey Hinton for foundational discoveries and inventions that enable machine ... The U.S. National Science Foundation congratulates John J. Hopfield and Geoffrey E. Hinton for their Nobel Prize in physics. The Royal Swedish Academy of Sciences has decided to award the Nobel Prize in Physics 2025 to John Clarke, Michel H. Devoret and John M. Martinis “for the ... The Royal Swedish Academy of Sciences has decided to award the 2024 Nobel Prize in Physics to John J. Hopfield and Geoffrey E. Hinton “for ... Geoffrey Hinton delivered his Nobel Prize banquet speech on December 10, 2024 at Stockholm City Hall. Nobel Prize in Physics ; ← 2024 · 2025 · 2026 → ; John Clarke, Michel H. Devoret, and John M. Martinis · John Bardeen (2) · nobelprize.org/physics

In [ ]:
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.agents import create_openai_tools_agent, AgentExecutor
from langchain_core.tools import tool
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_postgres import PGVector
from langchain_core.messages import AIMessage, HumanMessage
import asyncio
import os


load_dotenv()

# --- 1. LLM 설정 ---
# llm = ChatOpenAI(
#     model="gpt-4o-mini", 
#     temperature=0.5,
#     streaming=True
# )

# Google Gemini
llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0.5,
    streaming=True
)

# --- 2. Embedding 모델 설정 ---
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

# --- 3. PGVector 설정 ---
CONNECTION_STRING = f"postgresql+psycopg2://{os.getenv('DB_USER')}:{os.getenv('DB_PASS')}@{os.getenv('DB_HOST')}:{os.getenv('DB_PORT')}/{os.getenv('DB_NAME')}"
COLLECTION_NAME = "rag_example"

vector_store = PGVector(
    embeddings=embeddings,
    collection_name=COLLECTION_NAME,
    connection=CONNECTION_STRING,
    use_jsonb=True,
)

# --- 4. 벡터 검색 도구 정의 ---
@tool
def search_documents(query: str) -> str:
    """
    데이터베이스에서 벡터 유사도 검색을 수행합니다.
    사용자의 질문과 의미적으로 유사한 문서를 찾아 반환합니다.
    
    예시:
    - "2024년 영국 AI 산업 현황은?" 
    - "중국의 AI 플러스 정책은?"
    - "최근 공개된 AI 모델은?"
    """
    try:
        results = vector_store.similarity_search(query, k=5)
        
        if not results:
            return "검색 결과가 없습니다."
        
        formatted_results = []
        for i, doc in enumerate(results, 1):
            formatted_results.append(f"[문서 {i}]\n{doc.page_content}\n")
        
        return "\n".join(formatted_results)
    
    except Exception as e:
        return f"검색 중 오류 발생: {str(e)}"

@tool
def search_documents_with_score(query: str) -> str:
    """
    데이터베이스에서 벡터 유사도 검색을 수행하고 유사도 점수도 함께 반환합니다.
    검색 결과의 신뢰도를 확인하고 싶을 때 사용하세요.
    """
    try:
        results = vector_store.similarity_search_with_score(query, k=5)
        
        if not results:
            return "검색 결과가 없습니다."
        
        formatted_results = []
        for i, (doc, score) in enumerate(results, 1):
            formatted_results.append(
                f"[문서 {i}] (유사도: {score:.4f})\n{doc.page_content[:500]}...\n"
            )
        
        return "\n".join(formatted_results)
    
    except Exception as e:
        return f"검색 중 오류 발생: {str(e)}"

# --- 5. 도구 리스트 ---
tools = [search_documents, search_documents_with_score]

# --- 6. 멀티턴 대화를 위한 프롬프트 ---
prompt = ChatPromptTemplate.from_messages([
    ("system", """당신은 AI 산업 동향 분석 전문가입니다.

    **역할:**
    1. 사용자의 질문을 분석하여 벡터 검색이 필요한지 판단합니다.
    2. 이전 대화 내용을 참고하여 맥락을 이해합니다.
    3. 벡터 유사도 검색을 통해 관련 문서를 찾습니다.
    4. 검색된 정보를 바탕으로 정확하고 자연스러운 답변을 생성합니다.

    **데이터베이스 내용:**
    - SPRi AI Brief 2025년 10월호
    - AI 산업의 최신 동향 (정책, 기업, 기술, 인력/교육)
    - 각국의 AI 정책 및 규제
    - 주요 기업의 AI 모델 발표
    - AI 기술 연구 동향

    **작업 순서:**
    1. 이전 대화 맥락 파악
    2. 현재 질문 분석 (대명사나 축약 표현을 맥락에 맞게 해석)
    3. search_documents 도구로 관련 문서 검색
    4. 검색 결과를 바탕으로 정확한 답변 작성

    **중요:**
    - 이전 대화를 기억하고 연속적인 질문에 답변하세요
    - "그것", "그 회사", "더 자세히" 등의 표현은 이전 대화를 참고하세요
    - 검색된 문서 내용만을 기반으로 답변하세요
    - 답변 시 출처를 간단히 언급하세요
    """),
        MessagesPlaceholder(variable_name="chat_history"),
        ("human", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad")
    ])

# --- 7. 에이전트 생성 ---
agent = create_openai_tools_agent(llm, tools, prompt)
agent_executor = AgentExecutor(
    agent=agent, 
    tools=tools, 
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=5
)

# --- 8. 멀티턴 대화 함수 (실제 토큰 스트리밍) ---
async def run_multiturn_conversation():
    """멀티턴 대화를 지원하는 대화형 루프 (토큰 단위 스트리밍)"""
    chat_history = []
    
    print("=" * 70)
    print("Agentic RAG 시스템 시작 (멀티턴 대화 + 토큰 스트리밍)")
    print("=" * 70)
    print("\n종료: 'quit', 'exit', '종료' 입력")
    print("대화 초기화: 'reset', 'clear', '초기화' 입력")
    print("=" * 70)
    
    while True:
        # asyncio에서 input 사용
        user_input = await asyncio.to_thread(input, "\n질문: ")
        user_input = user_input.strip()
        
        if user_input.lower() in ['quit', 'exit', '종료']:
            print("시스템을 종료합니다.")
            break
        
        if user_input.lower() in ['reset', 'clear', '초기화']:
            chat_history = []
            print("대화 기록이 초기화되었습니다.")
            continue
        
        if not user_input:
            continue
        
        try:
            print("\n답변: ", end="", flush=True)
            
            full_response = ""
            
            # astream_events로 실제 토큰 단위 스트리밍(비동기)
            async for event in agent_executor.astream_events(
                {
                    "input": user_input,
                    "chat_history": chat_history
                },
                version="v2"
            ):
                kind = event["event"]
                
                # 도구 시작
                if kind == "on_tool_start":
                    tool_name = event["name"]
                    print(f"\n[{tool_name}] 검색 중...", end="", flush=True)
                
                # 도구 종료
                elif kind == "on_tool_end":
                    print(" 완료\n답변: ", end="", flush=True)
                
                # LLM 토큰 스트리밍 (최종 답변)
                elif kind == "on_chat_model_stream":
                    content = event["data"]["chunk"].content
                    if content:
                        print(content, end="", flush=True)
                        full_response += content
            
            print()  # 줄바꿈
            
            # 대화 기록에 추가
            chat_history.append(HumanMessage(content=user_input))
            chat_history.append(AIMessage(content=full_response))
            
        except Exception as e:
            print(f"\n오류 발생: {str(e)}")

# --- 9. 실행 ---
if __name__ == "__main__":
    # asyncio.run(run_multiturn_conversation())
    await run_multiturn_conversation()

E0000 00:00:1760592693.376379 1073101 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.


Agentic RAG 시스템 시작 (멀티턴 대화 + 토큰 스트리밍)

종료: 'quit', 'exit', '종료' 입력
대화 초기화: 'reset', 'clear', '초기화' 입력

답변: 

> Entering new None chain...


E0000 00:00:1760592717.186260 1073101 alts_credentials.cc:93] ALTS creds ignored. Not running on GCP and untrusted ALTS is not enabled.



Invoking: `search_documents` with `{'query': 'AI 시장 동향'}`



[search_documents] 검색 중...[문서 1]
2025년10월호인공지능 산업의 최신 동향

[문서 2]
SPRi AI Brief2025년 10월호
24
스탠포드⼤ 연구 결과, AI 노출도 높은 직종의 신입 근로자 고용 감소n스탠포드⼤의 분석에 따르면 미국 노동시장에서 AI 확산이 경력 초기 근로자들에게 불균형한 영향을 미치면서 AI 노출도가 높은 직업에서 경력 초기 근로자의 고용이 뚜렷하게 감소n이러한 결과는 경기 상황 요인을 배제하거나 표본 구성을 달리했을 때도 일관되게 나타나, 생성 AI 확산이 경력 초기 근로자들에게 부정적 고용효과를 유발하고 있음을 입증
KEY Contents

[문서 3]
£에이전틱 AI 조기 도입 기업의 88%가 생성 AI 활용 사례에서 ROI 달성n구글 클라우드(Google Cloud)가 전 세계 글로벌 대기업 대상의 설문조사 결과를 바탕으로 2025년 9월 4일 ‘2025년 AI 투자수익률(ROI of AI)’ 보고서를 발간∙전 세계 매출 1,000만 달러 이상 기업의 고위 임원 3,466명을 대상으로 진행된 이번 조사 결과, 전 세계 대기업에서 AI 에이전트 도입이 빠르게 확산하는 추세  ∙생성 AI를 사용하는 기업의 52%가 AI 에이전트도 적극 활용하고 있다고 답했으며, 39%의 응답자는 기업이 10개 이상의 AI 에이전트를 도입했다고 응답∙전체 산업에서 AI 에이전트 활용 분야는 고객 서비스와 경험(49%), 마케팅(46%), 보안 운영과 사이버보안(46%), 기술지원(45%), 제품 혁신과 디자인(43%), 생산성과 연구(43%) 순을 기록∙지역별로 AI 에이전트 활용의 우선순위는 다르게 나타났으며, 유럽 지역 응답자들은 AI 기반 기술지원을 우선하는 반면, 아시아태평양 응답자들은 고객 서비스를, 라틴아메리카 응답자들은 마케팅을 우선시∙에이전틱 AI 조기 도입 기업(전체 응답 기업의 13%)은 AI 